In [ ]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

# Parameter Configuration

In [ ]:
# network parameters
n_step = 100  # input data length, for 1 second data with sampling frequency 100Hz, n_step=100
n_width = 3  # each sensor has 3 axes
n_channel = 2  # 2 sensors
n_hidden = 128  # hidden layer neuron. Recommended values 32, 64, 128

# learning parameters
learning_rate = 0.0025  # recommended values 0.0025, 0.005, 0.01
l2_lambda = 0.002  # recommended values 0.00025, 0.0005, 0.001, 0.002
dropout_prob = 0.5
training_iters = 300  # 300 rounds
batch_size = 200
stop_loss = 0.15

# Network Sturcture

In [ ]:
# tf graph input
x = tf.placeholder("float", [None, n_step, n_width, n_channel])
y = tf.placeholder("float", [None, n_class])
keep_prob = tf.placeholder(tf.float32)

# define weights
weights = {
    # Hidden layer weights
    'hidden': tf.Variable(tf.random_normal([n_width * n_channel, n_hidden])),
    # 3*n_class inputs, n_class output(class prediction)
    'out': tf.Variable(tf.random_normal([n_hidden, n_class]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_class]))
}


def stackLSTM(x, weights, biases, keep_prob):
    with tf.variable_scope('trans'), tf.name_scope('trans'):
        x_trans = tf.reshape(x, [-1, n_step, n_width * n_channel])
        x_trans = tf.transpose(x_trans, [1, 0, 2])
        x_trans = tf.reshape(x_trans, [-1, n_width * n_channel])
        # Linear activation
        x_act = tf.nn.relu(
            tf.matmul(x_trans, weights['hidden']) + biases['hidden'])
        # Split data because rnn cell needs a list of inputs for the RNN inner loop
        x_split = tf.split(x_act, n_step, 0)
    with tf.variable_scope('stack_LSTM'), tf.name_scope('stack_LSTM'):
        # Define two stacked LSTM cells
        lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(
            n_hidden, forget_bias=1.0, state_is_tuple=True)
        lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(
            n_hidden, forget_bias=1.0, state_is_tuple=True)
        lstm_cells = tf.contrib.rnn.MultiRNNCell(
            [lstm_cell_1, lstm_cell_2], state_is_tuple=True)
        # Get LSTM cell output
        outputs, states = tf.contrib.rnn.static_rnn(
            lstm_cells, x_split, dtype=tf.float32)
    with tf.variable_scope('output'), tf.name_scope('output'):
        # Get last time step's output feature for a "many to one" style classifier
        lstm_last_output = outputs[-1]
        batchnorm = tf.contrib.layers.batch_norm(lstm_last_output)
        dropout = tf.layers.dropout(batchnorm, rate=keep_prob)
        out = tf.matmul(dropout, weights['out']) + biases['out']
    return out


pred = stackLSTM(x, weights, biases, keep_prob)

# Define loss and optimizer
regularization = l2_lambda * (tf.nn.l2_loss(
    weights['hidden']) + tf.nn.l2_loss(weights['out']))
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=pred, labels=y)) + regularization
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))